In [2]:
#!pip install neptune
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from medmnist import DermaMNIST
import random
import neptune
import itertools

In [5]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=2, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.25)

        # Calculate the flattened size
        # The input shape (3, 64, 64) corresponds to the RGB channels (3) and image dimensions (64x64), matching the DermaMNIST dataset.
        self.flattened_size = self.get_flattened_size((3, 64, 64))
        self.fc1 = nn.Linear(self.flattened_size, 64)
        self.fc2 = nn.Linear(64, 7)

    def get_flattened_size(self, input_shape):
        x = torch.zeros(1, *input_shape)
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        return x.view(1, -1).size(1)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for data, target in train_loader:
        # Ensure that the target is in the correct format (class indices) for compatibility with CrossEntropyLoss.
        data, target = data.to(device), target.to(device)
        target = target.long()  # Ensure target is of type torch.long
        if len(target.size()) > 1:
                target = target.squeeze()  # Remove extra dimensions

        #optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

# Evaluation function
def evaluate(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            target = target.long()  # Ensure target is of type torch.long
            if len(target.size()) > 1:
                target = target.squeeze()  # Remove extra dimensions
            outputs = model(data)
            test_loss += criterion(outputs, target).item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == target).sum().item()
    test_loss /= len(test_loader)
    accuracy = correct / len(test_loader.dataset)
    return test_loss, accuracy

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)  # Set your desired seed value here
    
# Hyperparameters and device setup
batch_size = 128
epochs = 50
learning_rate = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformations
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10),
    transforms.RandomVerticalFlip(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Data preparation
train_dataset = DermaMNIST(root='./data', split="train", download=True, transform=transform_train, size=64)
test_dataset = DermaMNIST(root='./data', split="test", download=True, transform=transform_test, size=64)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, loss, and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
for param in model.parameters():
    param.grad = None
    

# Training and testing loop
train_losses = []
test_losses = []
accuracies = []
patience = 3  # Number of epochs to wait for improvement
best_test_loss = float('inf')
patience_counter = 0

# Definiowanie zakresów hiperparametrów
param_space = {
    "batch_size": [32, 64, 128, 256],
    "learning_rate": [0.1, 0.01, 0.001, 0.0001],
    "dropout_rate": [0.25, 0.5]
}

# Funkcja do testowania hiperparametrów
def run_experiment(params):
    run = neptune.init_run(
        capture_hardware_metrics=False,
        project="my-uni-workspace/Podyplomowka",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiMWJlMWEzZS1iYTI2LTQ1MDgtOWI3NS02YzI3MjM5NzlkOTcifQ==",
        name="DermaMNIST-Experiment-{params}", 
        tags=["Dataset: DermaMNIST", "Hyperparameter Sweep", "Optimizer: AdamW"],
    )
    # Aktualizuj dropout w modelu
    model.dropout.p = params['dropout_rate']

    # Przygotuj dataloadery z nowym batch_size
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)

    # Aktualizuj optimizer z nowym learning_rate
    optimizer = optim.AdamW(model.parameters(), lr=params['learning_rate'])

    # Trening modelu
    best_test_loss = float('inf')
    for epoch in range(epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        test_loss, accuracy = evaluate(model, test_loader, criterion, device)

        run["train/loss"].append(train_loss)
        run["test/loss"].append(test_loss)
        run["test/accuracy"].append(accuracy)
        
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            torch.save(model.state_dict(), f"best_model_hyperparameter.pth")

    run.stop()
    return best_test_loss

# Iteracja przez wszystkie kombinacje hiperparametrów
best_params = None
best_loss = float('inf')

for params in itertools.product(*param_space.values()):
    param_dict = dict(zip(param_space.keys(), params))
    print(f"Running experiment with params: {param_dict}")
    loss = run_experiment(param_dict)
    if loss < best_loss:
        best_loss = loss
        best_params = param_dict

print(f"Best parameters: {best_params} with loss: {best_loss}")

Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz
Running experiment with params: {'batch_size': 32, 'learning_rate': 0.1, 'dropout_rate': 0.25}
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/my-uni-workspace/Podyplomowka/e/POD-16
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 3 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 3 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/my-uni-workspace/Podyplomowka/e/POD-16/metadata
Running experiment with params: {'batch_size': 32, 'learning_rate': 0.1, 'dropout_rate': 0.5}
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/my-uni-workspace/Podyplomowka/e/POD-17
[neptune] [info   ] Shutting down bac